In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sqlite3
import xml.etree.ElementTree as ET

from dateutil.parser import parse

In [ ]:
#Establishing Connection
conn = sqlite3.connect('vs.db')
cur = conn.cursor()

In [13]:
folderpath = './varfind/'
comp_filename = '20161117.txt'
dat_filename = 'varfind.dat'
img_filename = 'varfind.fts'
xml_filename = 'varfind.xml'
vardata = np.loadtxt(folderpath + comp_filename, delimiter=" ")
print(vardata)
indata = np.loadtxt(folderpath + dat_filename, delimiter=" ")
e = ET.parse(folderpath + xml_filename).getroot()

image_file = fits.open(folderpath+img_filename)
image_data = image_file[0].data
image_header = image_file[0].header

JD = image_header["JD"]
exposure = image_header["EXPTIME"]
filter_type = image_header["FILTER"]
date = parse(image_header["DATE-OBS"]).date()
location = '('+image_header["RA"]+', '+image_header["DEC"]+')'

#Creating Arrays for Data
JD = indata[:, 0]
mag_JD = indata[:, 1::2]
mag_star = np.transpose(mag_JD)
magerror_JD = indata[:, ::2]
magerror_JD = magerror_JD[:, 1:]

varJD = vardata[:, 0]
varmag_JD = vardata[:, 1::2]
varmag_star = np.transpose(varmag_JD)
varmagerror_JD = vardata[:, ::2]
varmagerror_JD = varmagerror_JD[:, 1:]

image_file.close()

[[  2.45771065e+06   9.99999900e+01   9.99999000e+00 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771066e+06  -1.93950000e+00   2.72000000e-03 ...,   9.99999000e+00
    3.62817000e+00   1.28200000e-02]
 [  2.45771066e+06  -1.94804000e+00   3.14000000e-03 ...,   9.99999000e+00
    3.67250000e+00   1.35500000e-02]
 ..., 
 [  2.45771087e+06  -1.92032000e+00   2.29000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -1.92863000e+00   2.30000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -1.93019000e+00   2.32000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]]


In [ ]:
#Saving FITS Image
fig = plt.figure(figsize=(12,12))

plt.imshow(image_data, cmap='gray')
plt.colorbar()

fig.suptitle('NGC 2180 Star Field | Exposure: ' + str(exposure) + '| Filter: ' + filter_type, fontsize=20)

fig.savefig('./lightcurves/'+'field' + str(date) + '.png')

plt.close()

### Inserting Stack of Images in VSDB
cur.execute('SELECT count(StackID) FROM StackTable')
res = cur.fetchone()
#Determining the StackId for the given stack
stackID = res[0] + 1

string = (stackID, date, filter_type, exposure, location)
cur.execute('INSERT INTO StackTable(StackID, Date, Filter, Exptime, Location) VALUES (?,?,?,?,?)', string)


conn.commit()

In [ ]:
#Assigning Ref coordinates to stars and storing in DB
for star in e.iter('s'):
    loc = '('+star.get('x') + ', ' + star.get('y')+')'
    string = (stackID, star.get('id'), loc)
    cur.execute('INSERT INTO StarTable(StackID, StarID, S_Location) VALUES (?,?,?)', string)

conn.commit()

In [ ]:
#Inserting Light Curves
for n in range(0, len(mag_star)):
    mag = mag_star[n].tolist()
    nJD = JD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)
        else:
            string = (n+1, mag[(len(mag_star[n]) - 1) - idx], nJD[(len(mag_star[n]) - 1) - idx])
            cur.execute('INSERT INTO S_LightCurve(StarID, JD, Star_Mag) VALUES (?,?,?)', string)

conn.commit()

In [18]:
from scipy import signal

for n in range(0, len(varmag_star)):
    mag = varmag_star[n].tolist()
    nJD = varJD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()
    
    f, Pxx_den = signal.periodogram(mag)
    fig = plt.figure()
    plt.semilogy(f, Pxx_den)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('Linear spectrum [V RMS]')
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date) + ' Periodogram', fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '_pdgm.png')
    plt.close()
    

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\ticker.py:2210: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


In [ ]:
#TRANSLATE XY COORDS TO RA DEC
#FIND PERIODS OF LCs

In [ ]:
#Pulling Lightcurve from DB
cur.execute('SELECT Star_Mag, JD FROM S_LightCurve WHERE StarID = 1;')
res = np.array(cur.fetchall())
print(res[:,:])

In [ ]:
#If a comp star does not have a data point, do you remove all data points for all comp stars or negate that star from the average for that point
def defCompStars(stars):
    comp_mag_list = [mag_star[s] for s in stars]
    comp_JD = JD.tolist()
    
    comp_mag = np.zeros_like(comp_mag_list[0])
    '''for s in stars:
        comp_mag = np.add(comp_mag, mag_star[s])
        
    comp_mag /= len(stars)
    #Is this step necessary
    comp_mag = np.ndarray.tolist(comp_mag)
    
    #Popping Comp Star Values where a comp is null
    for idx, val in enumerate(reversed(comp_mag)):
        del_el = False
        index = (len(mag_star[0]) - 1) - idx
        for n in stars:
            del_el = del_el or (mag_star[n][index] >= 99)
        if del_el:
            print(index)
            comp_mag.pop(index)
            comp_JD.pop(index)
    '''
    return comp_mag, comp_JD

In [ ]:
c_mag, c_JD = defCompStars([1, 2, 500])
print(len(c_mag))

In [ ]:
#Plotting the nth star's light curve
def plotLightCurve(n):
    
    mag = mag_star[n].tolist()
    nJD = JD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()

In [ ]:
plotLightCurve(516)

In [ ]:
#Closing the DB
cur.close()
conn.close()

# Links
https://d17h27t6h515a5.cloudfront.net/topher/2016/September/57ed880e_sql-sqlite-commands-cheat-sheet/sql-sqlite-commands-cheat-sheet.pdf
https://s3.amazonaws.com/michael-kennedy/downloads/DM/python_cheat_sheet.pdf


In [23]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from dateutil.parser import parse
from scipy import signal

folderpath = './varfind/'
comp_filename = '20161117.txt'
img_filename = 'varfind.fts'
vardata = np.loadtxt(folderpath + comp_filename, delimiter=" ")
print(vardata)

image_file = fits.open(folderpath+img_filename)
image_data = image_file[0].data
image_header = image_file[0].header

JD = image_header["JD"]
exposure = image_header["EXPTIME"]
filter_type = image_header["FILTER"]
date = parse(image_header["DATE-OBS"]).date()
location = '('+image_header["RA"]+', '+image_header["DEC"]+')'

#Creating Arrays for Data

varJD = vardata[:, 0]
varmag_JD = vardata[:, 1::2]
varmag_star = np.transpose(varmag_JD)
varmagerror_JD = vardata[:, ::2]
varmagerror_JD = varmagerror_JD[:, 1:]

image_file.close()

star_period_frames = [(0, 0, 0)]

for n in range(0, len(varmag_star)):
    mag = varmag_star[n].tolist()
    nJD = varJD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(mag_star[n]) - 1) - idx)
            nJD.pop((len(mag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()
    
    fs = 1/exposure
    f, Pxx_den = signal.periodogram(mag, fs)
    star_period_frames.append((n, np.amax(Pxx_den), len(mag)))
    fig = plt.figure()
    plt.semilogy(f, Pxx_den)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('Linear spectrum [V RMS]')
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date) + ' Periodogram', fontsize=18)
    fig.savefig('./lightcurves/'+'lc_' + str(n) + '_pdgm.png')
    plt.close()
    
print(star_period_frames)

[[  2.45771065e+06   9.99999900e+01   9.99999000e+00 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771066e+06  -1.93950000e+00   2.72000000e-03 ...,   9.99999000e+00
    3.62817000e+00   1.28200000e-02]
 [  2.45771066e+06  -1.94804000e+00   3.14000000e-03 ...,   9.99999000e+00
    3.67250000e+00   1.35500000e-02]
 ..., 
 [  2.45771087e+06  -1.92032000e+00   2.29000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -1.92863000e+00   2.30000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -1.93019000e+00   2.32000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]]


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\ticker.py:2210: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


[(0, 0, 0), (0, 0.042530207369081918, 91), (1, 0.026496581270688373, 91), (2, 0.043989579003320373, 91), (3, 0.018394805390587403, 91), (4, 0.016333483424770433, 91), (5, 0.023385019927474531, 91), (6, 0.021789296055791621, 91), (7, 0.014176375011887047, 91), (8, 0.015892619108207131, 91), (9, 0.037986555856157685, 91), (10, 0.023777509266835573, 91), (11, 0.022811444897322595, 91), (12, 0.016318701512167588, 91), (13, 0.034142351575689692, 91), (14, 0.022429955220102594, 91), (15, 0.030329735487726595, 91), (16, 0.056539694622892205, 91), (17, 0.050948315773255218, 91), (18, 0.034559085858184166, 91), (19, 0.02809443057068688, 91), (20, 0.012683486316424953, 21), (21, 0.014808031767066876, 91), (22, 0.056202287262935434, 91), (23, 0.069456137635049098, 88), (24, 0.029979744128906007, 91), (25, 0.10105964857434289, 91), (26, 0.019760380299193275, 91), (27, 0.012457907549766976, 91), (28, 0.070532636247421746, 91), (29, 0.026916747832161986, 91), (30, 0.023433095172275237, 91), (31, 0.0

In [26]:
period_star_frames = sorted(star_period_frames, key=lambda tup: tup[1], reverse = True)
print(period_star_frames)

[(230, 3.2721052794894034, 13), (211, 1.9464317417489676, 91), (161, 1.4962563151316142, 73), (80, 1.4043776882057379, 74), (310, 1.230173286736876, 70), (287, 0.90781032974243736, 70), (221, 0.7801400637047885, 90), (285, 0.76155064959199714, 87), (375, 0.69694164293337346, 48), (202, 0.62146015935941801, 91), (249, 0.55163587910191769, 85), (182, 0.53909069258947473, 91), (320, 0.52777455408391583, 85), (259, 0.52763088488334153, 87), (327, 0.52702011136539206, 66), (394, 0.50240306357166709, 28), (345, 0.49304311110043719, 75), (398, 0.48611123146432089, 26), (102, 0.47855018211397465, 88), (263, 0.47268534845940707, 85), (296, 0.4681421523676027, 79), (349, 0.46300516902905259, 67), (275, 0.43895180914565102, 79), (239, 0.43836175247932696, 88), (272, 0.43739976635221384, 80), (281, 0.42368317914367298, 81), (426, 0.42136388463636337, 33), (387, 0.41805549517346241, 15), (141, 0.41350114637872015, 90), (372, 0.41207236870891073, 42), (160, 0.4064018317982942, 91), (261, 0.405804676